# LangWatch DSPy Visualizer

This notebook shows an example of a simple DSPy optimization process integrated with LangWatch for training visualization and debugging.

[<img align="center" src="https://colab.research.google.com/assets/colab-badge.svg" />](https://colab.research.google.com/github/langwatch/langwatch/blob/main/python-sdk/examples/dspy_visualization.ipynb)

In [1]:
# Install langwatch along with dspy for the visualization
!pip install dspy langwatch


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## Preparing the LLM

In [2]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter your OPENAI_API_KEY: ")

import dspy

llm = dspy.LM("openai/gpt-4o-mini", api_key=os.environ["OPENAI_API_KEY"])

print("LLM test response:", llm("hello there"))

colbertv2_wiki17_abstracts = dspy.ColBERTv2(
    url="http://20.102.90.50:2017/wiki17_abstracts"
)
dspy.settings.configure(lm=llm, rm=colbertv2_wiki17_abstracts)

LLM test response: ['Hello! How can I assist you today?']


## Preparing the Dataset

In [3]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=32, eval_seed=2025, dev_size=50, test_size=0)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

(32, 50)

## Defining the model

In [4]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")


class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages # type: ignore
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)


dev_example = devset[18]
print(f"[Devset] Question: {dev_example.question}")
print(f"[Devset] Answer: {dev_example.answer}")
print(f"[Devset] Relevant Wikipedia Titles: {dev_example.gold_titles}")

generate_answer = RAG()

pred = generate_answer(question=dev_example.question)

# Print the input and the prediction.
print(f"[Prediction] Question: {dev_example.question}")
print(f"[Prediction] Predicted Answer: {pred.answer}")

[Devset] Question: Which magazine was released first, Fortune or Motor Trend?
[Devset] Answer: Motor Trend
[Devset] Relevant Wikipedia Titles: {'Fortune (magazine)', 'Motor Trend'}
[Prediction] Question: Which magazine was released first, Fortune or Motor Trend?
[Prediction] Predicted Answer: Fortune


## Login to LangWatch

In [1]:
import langwatch

langwatch.login()

/Users/rchaves/Projects/langwatch-saas/langwatch/langwatch_nlp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Please go to https://app.langwatch.ai/authorize to get your API key
LangWatch API key set


## Start Training Session!

In [6]:
from dspy.teleprompt import MIPROv2
import dspy.evaluate

# Define our metric validation
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

# Set up a MIPROv2 optimizer, which will compile our RAG program.
optimizer = MIPROv2(metric=validate_context_and_answer, prompt_model=llm, task_model=llm, num_candidates=2, init_temperature=0.7)

# Initialize langwatch for this run, to track the optimizer compilation
langwatch.dspy.init(experiment="my-awesome-experiment", optimizer=optimizer)

# Compile
compiled_rag = optimizer.compile( RAG(),
    trainset=trainset,
    num_trials=10,
    max_bootstrapped_demos=3,
    max_labeled_demos=5,
)

2025/05/09 14:34:42 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: False
num_candidates: 19
valset size: 25




[LangWatch] Experiment initialized, run_id: gay-jasmine-pigeon
[LangWatch] Open https://app.langwatch.ai/public-documentation-examples-YxS3Bf/experiments/my-awesome-experiment?runIds=gay-jasmine-pigeon to track your DSPy training session live



2025/05/09 14:34:44 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/05/09 14:34:44 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/05/09 14:34:44 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


100%|██████████| 7/7 [00:00<00:00, 216.70it/s]


...

2025/05/09 14:35:01 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 40.0 with parameters ['Predictor 0: Instruction 6', 'Predictor 0: Few-Shot Set 15'].
2025/05/09 14:35:01 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [40.0, 32.0, 36.0, 44.0, 36.0, 40.0, 36.0, 40.0, 40.0, 44.0, 44.0, 36.0, 44.0, 44.0, 44.0, 40.0, 40.0, 48.0, 40.0, 48.0, 40.0, 44.0, 48.0, 48.0, 40.0]
2025/05/09 14:35:01 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 48.0
2025/05/09 14:35:01 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/05/09 14:35:01 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 26 / 25 =====


Average Metric: 9.00 / 25 (36.0%): 100%|██████████| 25/25 [00:00<00:00, 377.49it/s]

2025/05/09 14:35:01 INFO dspy.evaluate.evaluate: Average Metric: 9 / 25 (36.0%)


2025/05/09 14:35:01 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 36.0 with parameters ['Predictor 0: Instruction 6', 'Predictor 0: Few-Shot Set 17'].
2025/05/09 14:35:01 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [40.0, 32.0, 36.0, 44.0, 36.0, 40.0, 36.0, 40.0, 40.0, 44.0, 44.0, 36.0, 44.0, 44.0, 44.0, 40.0, 40.0, 48.0, 40.0, 48.0, 40.0, 44.0, 48.0, 48.0, 40.0, 36.0]
2025/05/09 14:35:01 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 48.0
2025/05/09 14:35:01 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/05/09 14:35:01 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 48.0!


: 

In [11]:
compiled_rag

generate_answer.predict = Predict(StringSignature(context, question -> reasoning, answer
    instructions='Given the context and the question, generate a reasoned answer that is concise and factually correct, ensuring it consists of no more than 5 words.'
    context = Field(annotation=str required=True json_schema_extra={'desc': 'may contain relevant facts', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': 'often between 1 and 5 words', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
))

In [ ]:
compiled_rag.save("optimized_model.json")